## Chapter2: 시계열 데이터 수집 및 전처리

이번 장에서는 시계열 데이터 수집, 전처리하는 방법을 배운다.

### 목차
- 온라인 저장소에서 시계열 데이터 찾기
- 시계열을 고려하지 않고 수집된 데이터에서 시계열 데이터 수집 및 전처리
- timestamp 다루는 방법


### 2.1. 시계열 데이터 찾기

시계열 데이터는 목적에 따라 크게

1. 학습, 실험 목적으로 생성된 데이터
    - [UCI 머신러닝 저장소 - 시계열](https://archive.ics.uci.edu/ml/datasets.php?format=&task=&att=&area=&numAtt=&numIns=&type=ts&sort=nameUp&view=table): 결근, 공기질 등.
    - [UEA, UCR 시계열 분류 저장소](https://perma.cc/56Q5-YPNT): [요가 동작 분류 작업](https://perma.cc/U6MU-2SCZ), [와인 데이터셋](https://perma.cc/Y34-UGMD) 등
    - 정부 시계열 데이터셋: 미국 국립환경정보센터NCEI, 미국 노동통계국, 미국 질병통제에방센터CDC, 세인트루이스 연방준비은행, 한국 기상청 등. 경제와 기후, 범죄에 대한 정부 지출과 각 범죄율을 병렬로 다변량 시계열을 사용하면서 분석할 수 있다. 하지만 여러 요인이 작용하므로 학습용으로는 적합하지 않을 수 있음.
    - [CompEngine](https://www.comp-engine.org/#!browse/category): 시계열 데이터를 스스로 조직화하는 데이터베이스.
    - R의 Mcomp, M4comp2018 패키지, [CRAN 저장소](https://perma.cc/2694-D79K)의 시계열 데이터 섹션
2. 시계열이 아닌 데이터에서 시계열 데이터 생성: timestamp를 시계열로 변환 및 결합
    - timestamp가 찍힌 기록: 임의의 시점과 그 시점 이후의 시점 타임 스탬프(접근 등)으로 시간의 변화량을 모델링할 수 있음
    - 시간 없는 측정으로 시간 대체: 데이터가 시간을 명시적으로 포함하지 않아도, 데이터의 숨은 논리로 유추할 수 있는 경우. 센서 주기 등
    - 물리적 흔적: 디지털로 저장된 흔적을 이미지나 데이터베이스에 저장됨

으로 나눌 수 있다.


- 유의: 데이터에 구체적 '시간'이 없더라도, **일련의 정렬된 형태를 띈다면 시계열 적용 가능**






### 2.2. 흩어진 데이터를 시계열 데이터로 만들기

NGO에서 시계열 분석을 한다고 할 때, SQL로도 충분히 시계열에 대한 현황 분석을 할 수 있다.
- 메일을 읽었는지:  빈도, 응답 시간에 따라 메일 수신의 피로감 파악
- 기부 멤버십 중단 경험: 시계열 기부 예측 
- 거래 내역: 어떤 특정한 시점에 구매할지 예측

하지만 보통 데이터베이스 스키마를 설계할 때, 시계열 분석을 고려하지 않아 따로 시계열을 수집하고 구성해야한다.

아래 데이터의 시간 구분은 다음과 같다.
- YearJoined: 연간 회원 상태
- emails: 이메일 열람 관련 주간 누적 기록
- donations: 기부가 이루어진 순간의 타임스탬프

In [1]:
import pandas as pd
import io
import requests

In [4]:
YearJoined = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/year_joined.csv")
YearJoined.head()

,user,userStats,yearJoined
0,0,silver,2014
1,1,silver,2015
2,2,silver,2016
3,3,bronze,2018
4,4,silver,2018


In [6]:
emails = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/emails.csv")
emails.head()

,emailsOpened,user,week
0,3.0,1.0,2015-06-29 00:00:00
1,2.0,1.0,2015-07-13 00:00:00
2,2.0,1.0,2015-07-20 00:00:00
3,3.0,1.0,2015-07-27 00:00:00
4,1.0,1.0,2015-08-03 00:00:00


In [7]:
donations = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/donations.csv")
donations.head()

,amount,timestamp,user
0,25.0,2017-11-12 11:13:44,0.0
1,50.0,2015-08-25 19:01:45,0.0
2,25.0,2015-03-26 12:03:47,0.0
3,50.0,2016-07-06 12:24:55,0.0
4,50.0,2016-05-11 18:13:04,1.0


1000명의 모든 회원이 하나의 상태만 가진다. 따라서 year Joined는 가입연도와 현재 혹은 가입 당시의 상태를 나타낼 가능성이 높음. 정확하게 분석하려면 데이터 파이프라인을 알아야한다.

 - **사전관찰**: 과거 데이터 분석에 현재 회원의 상태를 적용하면 알 수 없는 시계열 모델에 무언가를 입력하는 꼴이 된다.
    - 시계열에서 사전관찰: 미래의 어떤 사실을 안다는 의미로 사용. 데이터를 통해 실제로 알아야하는 시점보다 더 일찍 미래에 대한 사실을 발견. 미래에 일어난 일에 대한 정보가 과거 모델 초기 동작에 영향을 주는 방법. 자동화된 코드나 통게 테스트가 없어서 스스로 고민해야함.


In [3]:
# 28 page
YearJoined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


- 고려해야할 점
    - 어떤 방식으로 시간을 표현했는지: 주wwek를 나눌 때 1.1로 나누는게 아니라 첫 월요일을 기준으로 하는 등.
    - null 값이 있는지: **시계열에서는 null이 존재해야한다**. 아무일도 발생하지 않는 주 또한 데이터의 일부.



In [9]:
emails[emails.emailsOpened < 1]

,emailsOpened,user,week


- 두 가지 가능성
    1. null이 발생하지 않았거나, 
    2. 모든 회원이 이메일을 열람했다는 이벤트가 적어도 하나 존재

하지만 모든 회원이 매주 이메일 열람은 가능성이 낮다. 따라서 특정 한 회원 데이터를 살펴보면서 판단 가능

In [12]:
emails[emails.user == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04 00:00:00
25465,3.0,998.0,2017-12-11 00:00:00
25466,3.0,998.0,2017-12-18 00:00:00
25467,3.0,998.0,2018-01-01 00:00:00
25468,3.0,998.0,2018-01-08 00:00:00
25469,2.0,998.0,2018-01-15 00:00:00
25470,3.0,998.0,2018-01-22 00:00:00
25471,2.0,998.0,2018-01-29 00:00:00
25472,3.0,998.0,2018-02-05 00:00:00
25473,3.0,998.0,2018-02-12 00:00:00


일부 주가 누락됨. 17년 12.18. 이후 이메일 열람 기록이 없음.

#### 수학적으로 주차 계산하기

In [15]:
emails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25488 entries, 0 to 25487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   emailsOpened  25488 non-null  float64
 1   user          25488 non-null  float64
 2   week          25488 non-null  object 
dtypes: float64(2), object(1)
memory usage: 597.5+ KB


In [16]:
import datetime
date_time_str_max = max(emails[emails.user == 998].week)
date_time_str_min = min(emails[emails.user == 998].week)

date_time_obj_max = datetime.datetime.strptime(date_time_str_max, '%Y-%m-%d %H:%M:%S')
date_time_obj_min = datetime.datetime.strptime(date_time_str_min, '%Y-%m-%d %H:%M:%S')

(date_time_obj_max - date_time_obj_min).days/7

25.0

In [17]:
emails[emails.user == 998].shape

(24, 3)

전체 26주 중 24개 데이터만 있는 것으로 보아 2개 누락. 물론 이는 한 명의 데이터이므로 다른 사람은 다를 수도 있다. null 값은 없지만, 아예 데이터가 수집되지 않은 주는 있다.

- 왜 25가 아니고 26주?
    - 마지막에 1을 더할지 말지 고민해야함
    - 4월 7, 14, 21, 28 -> 4일
    - (마지막 날 - 처음 날) / 7 = (28-7) / 7 = 3
    - 처음, 혹은 마지막 날을 빼먹음. 따라서 1 더해주어야 함.


#### 판다스 multi index를 이용해 주차 구하기

In [18]:
complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.user)))

In [19]:
complete_idx

MultiIndex([('2017-12-18 00:00:00',   1.0),
            ('2017-12-18 00:00:00',   3.0),
            ('2017-12-18 00:00:00',   5.0),
            ('2017-12-18 00:00:00',   6.0),
            ('2017-12-18 00:00:00',   9.0),
            ('2017-12-18 00:00:00',  10.0),
            ('2017-12-18 00:00:00',  14.0),
            ('2017-12-18 00:00:00',  16.0),
            ('2017-12-18 00:00:00',  20.0),
            ('2017-12-18 00:00:00',  21.0),
            ...
            ('2016-01-04 00:00:00', 973.0),
            ('2016-01-04 00:00:00', 977.0),
            ('2016-01-04 00:00:00', 982.0),
            ('2016-01-04 00:00:00', 984.0),
            ('2016-01-04 00:00:00', 987.0),
            ('2016-01-04 00:00:00', 991.0),
            ('2016-01-04 00:00:00', 992.0),
            ('2016-01-04 00:00:00', 993.0),
            ('2016-01-04 00:00:00', 995.0),
            ('2016-01-04 00:00:00', 998.0)],
           length=93247)

In [24]:
all_email = emails.set_index(['week', 'user']).reindex(complete_idx, fill_value=0).reset_index()
all_email.columns = ['week', 'user', 'emailsOpened']

In [25]:
all_email[all_email.user ==998].sort_values('week')

,week,user,emailsOpened
18864,2015-02-09 00:00:00,998.0,0.0
12935,2015-02-16 00:00:00,998.0,0.0
49587,2015-02-23 00:00:00,998.0,0.0
49048,2015-03-02 00:00:00,998.0,0.0
78154,2015-03-09 00:00:00,998.0,0.0
...,...,...,...
9162,2018-04-30 00:00:00,998.0,3.0
42580,2018-05-07 00:00:00,998.0,3.0
16169,2018-05-14 00:00:00,998.0,3.0
31800,2018-05-21 00:00:00,998.0,3.0
